In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder

In [2]:
#LOAD ORIGINAL DATA
path = 'adult19.csv'
interviewData = pd.read_csv (path, index_col = 'HHX')
interviewData

,URBRRL,RATCAT_A,INCGRP_A,INCTCFLG_A,FAMINCTC_A,IMPINCFLG_A,PPSU,PSTRAT,HISPALLP_A,RACEALLP_A,...,PHSTAT_A,PROXYREL_A,PROXY_A,AVAIL_A,HHSTAT_A,INTV_QRT,RECTYPE,WTFA_A,WTIA_A,POVRATTC_A
HHX,,,,,,,,,,,,,,,,,,,,,
H048109,4,9,3,0,60000,2,2,122,3,2,...,3,NaN,NaN,1,1,1,10,13177.008,7601.336,2.96
H027044,4,9,3,0,50000,0,2,122,2,1,...,1,NaN,NaN,1,1,1,10,6140.552,3344.434,2.97
H058855,4,12,3,0,65000,1,2,122,2,1,...,3,NaN,NaN,1,1,1,10,9191.061,6949.498,4.28
H031993,4,14,5,0,120000,0,2,122,1,8,...,2,NaN,NaN,1,1,1,10,7900.035,6446.327,7.13
H007122,1,4,1,0,30000,0,2,115,2,1,...,4,NaN,NaN,1,1,1,10,10875.772,8646.586,1.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
H046022,4,14,5,0,116204,0,100,114,2,1,...,3,NaN,NaN,1,1,4,10,2679.016,1965.416,7.65
H046232,4,8,3,0,68000,0,100,114,2,1,...,3,NaN,NaN,1,1,4,10,6726.495,3920.208,2.02
H043765,4,13,3,0,60000,0,100,114,2,1,...,2,NaN,NaN,1,1,4,10,1246.306,864.743,4.98


In [3]:
#LOAD CLEAN DATA
dataDir = 'CleanDataFinal'

#LOAD TESTING DATA
#features
path = os.path.join(dataDir, 'testFeaturesFinal.csv')
test_x_pd = pd.read_csv (path, index_col = 'HHX')
test_x = test_x_pd.to_numpy()
print(test_x.shape)

(5556, 5290)


## GET TESTING DATA IN ORIGINAL FORMAT (NOT ONE-HOT ENCODED)

In [4]:
#LOAD CSV WITH DATA CLEANING INSTRUCTIONS
path = 'VariableNanDetails.csv'
data_cleaning_inst = pd.read_csv (path)
data_cleaning_inst

,COLUMN_INDEX,COLUMN_NAME,DESCRIPTION,DATA_TYPE,CATEGORICAL_TYPE,NANs,REPLACE_WITH,DROP_NAN_CODE_1,DROP_NAN_CODE_2,DROP_NAN_CODE_3,...,Code_4_Meaning,Code_5,Code_5_Meaning,Code_6,Code_6_Meaning,Code_7,Code_7_Meaning,Code_8,Code_8_Meaning,NOTES
0,402.0,POVRATTC_A,SA family poverty ratio,numerical,NaN,drop_nans,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,403.0,HHX,Randomly assigned household number unique to a...,recordkeeping - ID,NaN,drop_nans,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,404.0,WTIA_A,Weight - annual pre-post stratification calibr...,recordkeeping,NaN,drop_col,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,405.0,WTFA_A,Weight - Final Annual,recordkeeping,NaN,drop_col,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,406.0,RECTYPE,Record type,recordkeeping,NaN,drop_col,NaN,NaN,NaN,NaN,...,Sample Child Income,50.0,Paradata,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529,264.0,PHQ82_A,"How often feeling down, past 2 weeks",categorical,ordinal,drop_col,NaN,NaN,NaN,NaN,...,Nearly every day,7.0,Refused,8.0,Not Ascertained,9.0,Don't Know,NaN,NaN,PHQ questions
530,265.0,PHQ81_A,"How often little interest in things, past 2 weeks",categorical,ordinal,drop_col,NaN,NaN,NaN,NaN,...,Nearly every day,7.0,Refused,8.0,Not Ascertained,9.0,Don't Know,NaN,NaN,PHQ questions
531,266.0,MHTHND_A,Needed counseling/therapy but did not get it d...,categorical,nominal,drop_nans,NaN,NaN,NaN,NaN,...,Nearly every day,7.0,Refused,8.0,Not Ascertained,9.0,Don't Know,NaN,NaN,NaN
532,267.0,MHTHDLY_A,"Delayed counseling/therapy due to cost, past 12m",categorical,nominal,drop_nans,NaN,NaN,NaN,NaN,...,Nearly every day,7.0,Refused,8.0,Not Ascertained,9.0,Don't Know,NaN,NaN,NaN


In [5]:
#GRAB TESTING DATA USING HHX IN ITS ORIGINAL FORMAT
testHHX = test_x_pd.index.tolist()
test_data_original = interviewData.loc[testHHX, :]
#put HHX in as column (rest of script expects it as a column, not index)
test_data_original=test_data_original.reset_index()
test_data_original = test_data_original.rename(columns = {'index':'HHX'})
test_data_original

,HHX,URBRRL,RATCAT_A,INCGRP_A,INCTCFLG_A,FAMINCTC_A,IMPINCFLG_A,PPSU,PSTRAT,HISPALLP_A,...,PHSTAT_A,PROXYREL_A,PROXY_A,AVAIL_A,HHSTAT_A,INTV_QRT,RECTYPE,WTFA_A,WTIA_A,POVRATTC_A
0,H038561,1,14,5,0,200000,0,63,101,2,...,2,NaN,NaN,1,1,4,10,11037.472,5225.924,10.00
1,H043980,4,14,5,1,220000,0,16,109,2,...,1,NaN,NaN,1,1,4,10,6073.945,5716.518,10.00
2,H052102,3,7,1,0,25000,0,5,126,2,...,1,NaN,NaN,1,1,2,10,4454.983,3414.149,1.91
3,H015803,3,14,5,1,220000,0,4,100,2,...,2,NaN,NaN,1,1,3,10,9778.142,11062.271,10.00
4,H000267,2,13,4,0,96000,0,3,117,3,...,3,1.0,1.0,3,1,2,10,9386.832,10813.146,4.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5551,H035143,1,12,3,0,55000,2,6,113,2,...,2,NaN,NaN,1,1,2,10,6309.642,6385.852,4.21
5552,H035351,1,14,5,0,200000,0,23,133,2,...,1,NaN,NaN,1,1,3,10,12880.432,12987.214,10.00
5553,H019908,3,14,5,0,155000,0,26,118,2,...,1,NaN,NaN,1,1,2,10,4631.288,5408.342,9.21
5554,H038324,1,7,2,0,49000,0,25,135,1,...,1,NaN,NaN,1,1,2,10,12084.433,10930.770,1.89


In [6]:
#DROP RECORDKEEPING COLUMNS

#make working copy of data
cleanData = test_data_original.copy()
#make list of columns to drop 
cols = data_cleaning_inst.COLUMN_NAME[(data_cleaning_inst.DATA_TYPE == 'recordkeeping') &
                                      (data_cleaning_inst.NANs == 'drop_col')].tolist()
# drop irrelevant columns
cleanData = cleanData.drop(columns = cols)

#list columns that were dropped
print(data_cleaning_inst[['COLUMN_NAME', 'DESCRIPTION']][data_cleaning_inst.COLUMN_NAME.isin(cols)].to_string())
#print clean shape
print(cleanData.shape)

      COLUMN_NAME                                                                       DESCRIPTION
2          WTIA_A                               Weight - annual pre-post stratification calibration
3          WTFA_A                                                             Weight - Final Annual
4         RECTYPE                                                                       Record type
6        HHSTAT_A                                              Indicates person is the Sample Adult
137   IMPINCFLG_A                                         Imputed SA family income imputation flag 
138          PPSU                                Pseudo-PSU for public-use file variance estimation
139        PSTRAT                     \r\nPseudo-stratum for public-use file variance estimation \r
165  HHRESPSA_FLG  Sample Adult is the household respondent or the proxy who lives in the household
194       OGFLG_A                                                Other government reassignment flag


In [7]:
#DROP PHQ COLUMNS

#make list of columns to drop 
cols = data_cleaning_inst.COLUMN_NAME[(data_cleaning_inst.COLUMN_NAME.str.contains('PHQ', case=False)) &
    (data_cleaning_inst.NANs == 'drop_col')].tolist()

# drop irrelevant columns
cleanData = cleanData.drop(columns = cols)

#list columns that were dropped
print(data_cleaning_inst[['COLUMN_NAME', 'DESCRIPTION']][data_cleaning_inst.COLUMN_NAME.isin(cols)].to_string())
#print clean shape
print(cleanData.shape)

    COLUMN_NAME                                              DESCRIPTION
523     PHQ88_A  How often moving or speaking slow or fast, past 2 weeks
524     PHQ87_A            How often trouble concentrating, past 2 weeks
525     PHQ86_A           How often feeling bad about self, past 2 weeks
526     PHQ85_A        How often undereating or overeating, past 2 weeks
527     PHQ84_A                    How often feeling tired, past 2 weeks
528     PHQ83_A            How often trouble with sleeping, past 2 weeks
529     PHQ82_A                     How often feeling down, past 2 weeks
530     PHQ81_A        How often little interest in things, past 2 weeks
(5556, 507)


In [8]:
#DROP ROWS WITH NANS

#make list of columns to look for nans in 
cols = data_cleaning_inst.COLUMN_NAME[data_cleaning_inst.NANs.isin(['drop_nans', 'drop_nan_and_nan_codes'])]
#count nans per col
nans_drop_nans = cleanData[cols].isnull().sum(axis = 0)
    
# #drop rows with nans in cols
cleanData = cleanData.dropna(subset=cols)

#show nans per col
print(nans_drop_nans.to_string())
#print clean shape
print(cleanData.shape)        

POVRATTC_A      0
HHX             0
INTV_QRT        0
AVAIL_A         0
HYPEV_A         0
CHLEV_A         0
CHDEV_A         0
ASEV_A          0
CANEV_A         0
PREDIB_A        0
DIBEV_A         0
COPDEV_A        0
ARTHEV_A        0
DEMENEV_A       0
ANXEV_A         0
DEPEV_A         0
WEARGLSS_A      0
HEARAID_A       0
SOCWRKLIM_A     0
URBRRL          0
RATCAT_A        0
INCGRP_A        0
FAMINCTC_A      0
HISPALLP_A      0
RACEALLP_A      0
DISAB3_A        0
CITZNSTP_A      0
LEGMSTAT_A      0
MARSTAT_A       0
SMKECIGST_A     0
SMKCIGST_A      0
OTHGOV_A        0
OTHPUB_A        0
IHS_A           0
MILITARY_A      0
CHIP_A          0
MEDICAID_A      0
MEDICARE_A      0
PRIVATE_A       0
EDUC_A          0
MAXEDUC_A       0
PARSTAT_A       0
SAPARENTSC_A    0
MLTFAMFLG_A     0
OVER65FLG_A     0
HISDETP_A       0
HISP_A          0
REGION          0
MHTHRPY_A       0
DEPMED_A        0
DEPFREQ_A       0
ANXMED_A        0
ANXFREQ_A       0
HOMEHC12M_A     0
THERA12M_A      0
EYEEX12M_A

In [9]:
#CATEGORICAL ORDINAL DATA WITH NO 'NOT APPLICABLE' RESPONDENTS
#remove 'missing data codes' rows
#eg - not ascertained, refused, don't know

#check all columns with 'drop_nan_codes' instructions have missing data codes 
#data columns that need missing data codes remove
miss_code_col_A = data_cleaning_inst.COLUMN_NAME[data_cleaning_inst.NANs.isin(
    ['drop_nan_codes', 'drop_nan_and_nan_codes'])]

#instruction columns with missing data codes
miss_code_cols = ['DROP_NAN_CODE_1', 'DROP_NAN_CODE_2', 'DROP_NAN_CODE_3', 'DROP_NAN_CODE_4']
#select cols with missing data codes
miss_code_col_B = data_cleaning_inst.dropna(subset=miss_code_cols, how='all').COLUMN_NAME

#check they they are the same
miss_code_col_A = miss_code_col_A.sort_values()
miss_code_col_B = miss_code_col_B.sort_values()

#they are the same
print('all relevant cols have missing codes:', miss_code_col_A.equals(miss_code_col_B))

#make a copy (for ease in troubleshooting)
cleanData_copy = cleanData.copy()

#delete any rows with missing data codes in these rows
miss_code_col = miss_code_col_A.to_list()
for c in miss_code_col:
    miss_data_codes = data_cleaning_inst[miss_code_cols][data_cleaning_inst.COLUMN_NAME == c].dropna(axis=1).to_numpy().flatten()
    cleanData_copy = cleanData_copy[cleanData_copy[c].isin(miss_data_codes) == False]   

#check no missing codes remain     
miss_codes_remaining = []
for c in miss_code_col:
    unique_vals = np.sort(cleanData_copy[c].unique())
    miss_data_codes = np.sort(data_cleaning_inst[miss_code_cols][data_cleaning_inst.COLUMN_NAME == c].dropna(axis=1).to_numpy().flatten())
    intersection = np.intersect1d(unique_vals, miss_data_codes)
    miss_codes_remaining.append(intersection)   

print('missing codes remaining: ', np.sum(np.asarray(miss_codes_remaining)))

#save cleanData_copy as copy
cleanData = cleanData_copy.copy()
#look at shape 
print(cleanData.shape)

all relevant cols have missing codes: True
missing codes remaining:  0.0
(5556, 507)


In [10]:
#CATEGORICAL DATA: ADD NOT APPLICABLE CODES

#check all columns with 'replace' instructions have replace_with values
#data columns that need NaNs replaced
replaceNaNs_cols_A = data_cleaning_inst.COLUMN_NAME[data_cleaning_inst.NANs=='replace']

#select cols with replace-with values
replaceNaNs_cols_B = data_cleaning_inst.COLUMN_NAME[data_cleaning_inst.REPLACE_WITH.notna()]

#check they they are the same
replaceNaNs_cols_A = replaceNaNs_cols_A.sort_values()
replaceNaNs_cols_B = replaceNaNs_cols_B.sort_values()

#they are the same
print('all relevant cols have missing codes:', replaceNaNs_cols_A.equals(replaceNaNs_cols_B))

#make a copy (for ease in troubleshooting)
cleanData_copy = cleanData.copy()

#replace NaNs with appropriate code
replaceNaNs_cols = replaceNaNs_cols_A.to_list()
for c in replaceNaNs_cols:
    replace_code = data_cleaning_inst.REPLACE_WITH[data_cleaning_inst.COLUMN_NAME == c].to_numpy()[0]
    cleanData_copy[c] = cleanData_copy[c].fillna(replace_code)
    
#CHECK THERE ARE NO NANs LEFT
print('Number NaNs left: ', cleanData_copy.isnull().sum().sum())

#save cleanData_copy as copy
test_x_original = cleanData_copy.copy()
#set HHX back as index
test_x_original=test_x_original.set_index('HHX')
#look at shape 
print(test_x_original.shape)

all relevant cols have missing codes: True
Number NaNs left:  0
(5556, 506)


## REMOVE DATA RANDOMLY

In [11]:
# FUNCTION TO REMOVE DATA RANDOMLY
def remove_random_data(complete_data, data_percent):
    '''replaces random entries (single features from a row) with NaN.
    complete_data: array containing full dataset
    data_percent: integer indicating what percent of data to remove
    returns: array containing dataset with missing data (NaNs)'''
    #make copy of array 
    full_data = np.copy(complete_data)
    original_shape = full_data.shape
    #flatten data
    full_data = full_data.flatten()
    #make array for incomplete data
    inc_data = np.copy(full_data)

    #calculate number of entries to replace
    entries_to_remove = int(full_data.size*data_percent/100)

    #choose random indeces of data to replace
    i = np.random.choice(full_data.shape[0], entries_to_remove, replace=False)
    #replace random data
    for index in i:
        inc_data[index] = np.nan
    #reshape data back to original shape
    inc_data = np.reshape(inc_data, original_shape)
    return(inc_data)

## DATA IMPUTATION

In [12]:
#remove 15% of data
data_percent = 15
test_x_inc = remove_random_data(test_x_original, data_percent)
#turn to dataframe
test_x_inc_pd = pd.DataFrame(test_x_inc, columns=test_x_original.columns, index=test_x_original.index)
test_x_inc_pd

,URBRRL,RATCAT_A,INCGRP_A,INCTCFLG_A,FAMINCTC_A,HISPALLP_A,RACEALLP_A,DISAB3_A,SCHDYMSSTC_A,AFNOW,...,HYPMED_A,HYP12M_A,HYPDIF_A,HYPEV_A,PHSTAT_A,PROXYREL_A,PROXY_A,AVAIL_A,INTV_QRT,POVRATTC_A
HHX,,,,,,,,,,,,,,,,,,,,,
H038561,1.0,14.0,5.0,0.0,200000.0,2.0,1.0,2.0,100.0,2.0,...,NaN,10.0,10.0,2.0,2.0,NaN,10.0,NaN,NaN,10.00
H043980,4.0,14.0,5.0,1.0,NaN,2.0,1.0,2.0,100.0,10.0,...,NaN,10.0,10.0,2.0,1.0,NaN,10.0,1.0,4.0,NaN
H052102,3.0,7.0,1.0,0.0,25000.0,2.0,1.0,2.0,100.0,2.0,...,10.0,10.0,10.0,NaN,1.0,10.0,10.0,1.0,2.0,1.91
H015803,NaN,14.0,5.0,1.0,NaN,2.0,1.0,2.0,100.0,NaN,...,1.0,NaN,NaN,1.0,NaN,NaN,10.0,NaN,3.0,NaN
H000267,NaN,13.0,4.0,0.0,96000.0,3.0,NaN,2.0,NaN,2.0,...,NaN,10.0,10.0,2.0,3.0,1.0,NaN,3.0,2.0,4.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
H035143,1.0,12.0,3.0,0.0,55000.0,2.0,1.0,2.0,100.0,2.0,...,10.0,10.0,10.0,2.0,2.0,NaN,10.0,1.0,2.0,NaN
H035351,1.0,14.0,5.0,0.0,NaN,2.0,1.0,2.0,100.0,2.0,...,10.0,NaN,10.0,2.0,1.0,10.0,10.0,1.0,NaN,10.00
H019908,3.0,14.0,5.0,0.0,155000.0,2.0,1.0,2.0,100.0,2.0,...,10.0,10.0,10.0,2.0,1.0,10.0,10.0,NaN,2.0,9.21


In [13]:
#ISOLATE NUMERICAL AND CATEGORICAL FEATURES
#numerical features
col_num = data_cleaning_inst.COLUMN_NAME[
    (data_cleaning_inst.DATA_TYPE=='numerical') & (data_cleaning_inst.NANs != 'drop_col')]
data_num = test_x_inc_pd.loc[:, col_num]
#categorical features
data_cat = test_x_inc_pd.drop(columns=col_num)

## UNIVARIATE IMPUTATION

In [14]:
#NUMERICAL DATA
#turn to array
data_num_array = data_num.to_numpy()
#create imputer object
num_imp = SimpleImputer(missing_values=np.nan, strategy='median')
#fit simple imputer
num_imp.fit(data_num_array)
#impute missing data
data_num_imp = num_imp.transform(data_num_array)
#turn to dataframe
data_num_imp = pd.DataFrame(data_num_imp, columns=data_num.columns)
data_num_imp

,POVRATTC_A,INTV_QRT,FAMINCTC_A,AGEP_A
0,10.00,2.0,200000.0,27.0
1,3.49,4.0,60000.0,69.0
2,1.91,2.0,25000.0,61.0
3,3.49,3.0,60000.0,53.0
4,4.88,2.0,96000.0,85.0
...,...,...,...,...
5551,3.49,2.0,55000.0,43.0
5552,10.00,2.0,60000.0,35.0
5553,9.21,2.0,155000.0,54.0
5554,1.89,2.0,49000.0,22.0


In [15]:
#CATEGORICAL DATA
#turn to array
data_cat_array = data_cat.to_numpy()
#create imputer object
cat_imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
#fit simple imputer
cat_imp.fit(data_cat_array)
#impute missing data
data_cat_imp = cat_imp.transform(data_cat_array)
#turn to dataframe
data_cat_imp = pd.DataFrame(data_cat_imp, columns=data_cat.columns)
data_cat_imp

,URBRRL,RATCAT_A,INCGRP_A,INCTCFLG_A,HISPALLP_A,RACEALLP_A,DISAB3_A,SCHDYMSSTC_A,AFNOW,PHQCAT_A,...,CHL12M_A,CHLEV_A,HYPMED_A,HYP12M_A,HYPDIF_A,HYPEV_A,PHSTAT_A,PROXYREL_A,PROXY_A,AVAIL_A
0,1.0,14.0,5.0,0.0,2.0,1.0,2.0,100.0,2.0,1.0,...,10.0,2.0,10.0,10.0,10.0,2.0,2.0,10.0,10.0,1.0
1,4.0,14.0,5.0,1.0,2.0,1.0,2.0,100.0,10.0,1.0,...,2.0,1.0,10.0,10.0,10.0,2.0,1.0,10.0,10.0,1.0
2,3.0,7.0,1.0,0.0,2.0,1.0,2.0,100.0,2.0,1.0,...,1.0,1.0,10.0,10.0,10.0,2.0,1.0,10.0,10.0,1.0
3,3.0,14.0,5.0,1.0,2.0,1.0,2.0,100.0,2.0,1.0,...,1.0,1.0,1.0,10.0,10.0,1.0,2.0,10.0,10.0,1.0
4,3.0,13.0,4.0,0.0,3.0,1.0,2.0,100.0,2.0,1.0,...,10.0,2.0,10.0,10.0,10.0,2.0,3.0,1.0,10.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5551,1.0,12.0,3.0,0.0,2.0,1.0,2.0,100.0,2.0,1.0,...,10.0,2.0,10.0,10.0,10.0,2.0,2.0,10.0,10.0,1.0
5552,1.0,14.0,5.0,0.0,2.0,1.0,2.0,100.0,2.0,1.0,...,10.0,2.0,10.0,10.0,10.0,2.0,1.0,10.0,10.0,1.0
5553,3.0,14.0,5.0,0.0,2.0,1.0,2.0,100.0,2.0,1.0,...,10.0,2.0,10.0,10.0,10.0,2.0,1.0,10.0,10.0,1.0
5554,1.0,7.0,2.0,0.0,1.0,1.0,2.0,100.0,2.0,1.0,...,10.0,2.0,10.0,10.0,10.0,2.0,1.0,10.0,10.0,1.0


In [16]:
#ONE-HOT ENCODE CATEGORICAL DATA
#make list of columns to one-hot encode
cols = data_cleaning_inst.COLUMN_NAME[(data_cleaning_inst.ENCODING == 'one_hot') & 
                                      (data_cleaning_inst.COLUMN_NAME != 'PHQCAT_A')].tolist()
#isolate data to encode as one-hot
clean_onehot_data = data_cat_imp.loc[:, cols]
#create object
enc = OneHotEncoder()
#fit encoder
enc.fit(clean_onehot_data)
#transform data
clean_onehot_data = enc.transform(clean_onehot_data).toarray()
#get name of new columns
onehot_features = enc.get_feature_names_out(cols)
#join with previous data 
onehot_df = pd.DataFrame(clean_onehot_data, columns = onehot_features)
test_x_imp = pd.concat([data_num_imp, onehot_df], axis=1)
test_x_imp.index = test_x_inc_pd.index
test_x_imp.shape

(5556, 3524)

In [17]:
#STORE DATA

#make directory 
dataDir = 'ImputedData'
if not os.path.exists(dataDir):
    os.mkdir(dataDir)

#store imputed testing data
filepath = os.path.join(dataDir, 'SimpleImputedFeatures_05.csv')
test_x_imp.to_csv(filepath) 